In [1]:
from classes.PostgresConnect import PostgresConnect
import os

In [2]:
if(not os.path.exists('./csv/filtered/communes.csv')):
    %run './remodelisations/communes.ipynb'

if(not os.path.exists('./csv/filtered/departements.csv') or not os.path.exists('./csv/filtered/departements_cheflieu.csv')):
    %run './remodelisations/dep.ipynb'

if(not os.path.exists('./csv/filtered/regions.csv') or not os.path.exists('./csv/filtered/regions_cheflieu.csv')):
    %run './remodelisations/regions.ipynb'

if(not os.path.exists('./csv/filtered/indicateurs.csv') or not os.path.exists('./csv/filtered/statistiques.csv')):
    %run './remodelisations/statistiques.ipynb'


In [3]:
postgres = PostgresConnect()
postgres.connect()

In [4]:
tables = ['departements_cheflieu','regions_cheflieu', 'communes', 'departements', 'regions','indicateurs','statistiques']

In [5]:
# A enlever avant rendu 

drop_tables = ''
for table in tables:
    drop_tables += f'DROP TABLE IF EXISTS {table} CASCADE;'


In [6]:
postgres.sql(drop_tables)

In [7]:
postgres.sql(open("sql/create.sql", "r").read())

In [8]:
files = [
        'regions', 
        'departements',
        'communes',
        'departements_cheflieu',
        'regions_cheflieu',
        'indicateurs',
        'statistiques'
    ]

for file in files:
    path = f'./csv/filtered/{file}.csv'
    f = open(path, 'r')
    postgres.copy_from(f, file, ',', 'None', commit=True)
    f.close()


In [9]:
postgres.sql('select count(*) from communes;').all()

[(34965,)]

In [10]:
postgres.close()